1. Importação da lib Pandas para manipulação e pré-processamento dos dados, bem como o carregamento dos dados a serem analisados em DataFrames específicos.

In [ ]:
import pandas as pd

# Basta fazer o download dos microdados no seguinte endereço: https://download.inep.gov.br/microdados/microdados_enem_2024.zip
# Ambos os arquivos estão na pasta 'DADOS'.

df_participantes = pd.read_csv("PARTICIPANTES_2024.csv", sep=";", encoding="latin1", low_memory=False)
df_resultados = pd.read_csv("RESULTADOS_2024.csv", sep=";", encoding="latin1", low_memory=False)

2. df_participantes. Remoção de colunas irrelevantes para a análise, bem como a renomeação das colunas remanescentes.

In [ ]:
df_participantes = df_participantes.drop(columns=['NU_INSCRICAO', 'NU_ANO', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
                                                  'Q003', 'Q004', 'Q005', 'Q006', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020',
                                                  'Q021', 'Q022'])

df_participantes = df_participantes.rename(columns={
    "TP_FAIXA_ETARIA": "faixa_etaria",
    "TP_SEXO": "sexo",
    "TP_ESTADO_CIVIL": "estado_civil",
    "TP_COR_RACA": "cor_raca",
    "NO_MUNICIPIO_PROVA": "municipio_prova",
    "SG_UF_PROVA": "uf_prova",
    "Q001": "q_escolaridade_pai",
    "Q002": "q_escolaridade_mae",
    "Q007": "q_renda_familiar",
    "Q023": "q_tipo_em"
})

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em
0,5,F,1,1,Porto Alegre,RS,F,F,C,A
1,11,F,1,1,São Luiz Gonzaga,RS,B,C,E,A
2,11,F,1,1,Sarandi,RS,H,F,J,A
3,3,F,1,3,Novo Hamburgo,RS,B,B,C,A
4,16,M,3,1,Gravataí,RS,B,B,A,A


3. Redução da dimensionalidade das faixas etárias e criação da coluna "faixa_etaria_labels".

In [ ]:
def reduzir_faixa_etaria(x):
    if x == 1:
        return "Até 16"
    elif x == 2:
        return "17"
    elif x in [3, 4, 5]:
        return "18-20"
    elif x in [6, 7, 8, 9, 10]:
        return "21-25"
    elif x == 11:
        return "26-30"
    elif x in [12, 13]:
        return "31-40"
    elif x in [14, 15]:
        return "41-50"
    elif x in [16, 17]:
        return "51-60"
    else:  # 18, 19, 20
        return "60+"

# Criar a coluna com labels
df_participantes["faixa_etaria_labels"] = df_participantes["faixa_etaria"].apply(reduzir_faixa_etaria)

# Criar um dicionário para mapear labels → números
mapa_codigos = {
    "Até 16": 1,
    "17": 2,
    "18-20": 3,
    "21-25": 4,
    "26-30": 5,
    "31-40": 6,
    "41-50": 7,
    "51-60": 8,
    "60+": 9
}

# Substituir a coluna numérica original
df_participantes["faixa_etaria"] = df_participantes["faixa_etaria_labels"].map(mapa_codigos)

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels
0,3,F,1,1,Porto Alegre,RS,F,F,C,A,18-20
1,5,F,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30
2,5,F,1,1,Sarandi,RS,H,F,J,A,26-30
3,3,F,1,3,Novo Hamburgo,RS,B,B,C,A,18-20
4,8,M,3,1,Gravataí,RS,B,B,A,A,51-60


5. Transformação das strings "F"/"M" na coluna "sexo" em inteiros 0/1, além da cração da coluna "sexo_labels", com os rótulos dos números.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_participantes["sexo"] = le.fit_transform(df_participantes["sexo"])

mapa_sexo = {
    0: "Feminino",
    1: "Masculino"
}

df_participantes["sexo_labels"] = df_participantes["sexo"].map(mapa_sexo)

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels,sexo_labels
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,18-20,Feminino
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30,Feminino
2,5,0,1,1,Sarandi,RS,H,F,J,A,26-30,Feminino
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,18-20,Feminino
4,8,1,3,1,Gravataí,RS,B,B,A,A,51-60,Masculino


6. Criação da coluna "estado_civil_labels" para traduzir os números da coluna "estado_civil".

In [ ]:
mapa_estado_civil = {
    0: "Não informado",
    1: "Solteiro(a)",
    2: "Casado(a)/Mora com companheiro(a)",
    3: "Divorciado(a)/Desquitado(a)/Separado(a)",
    4: "Viúvo(a)"
}

df_participantes["estado_civil_labels"] = df_participantes["estado_civil"].map(mapa_estado_civil)

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels,sexo_labels,estado_civil_labels
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,18-20,Feminino,Solteiro(a)
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30,Feminino,Solteiro(a)
2,5,0,1,1,Sarandi,RS,H,F,J,A,26-30,Feminino,Solteiro(a)
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,18-20,Feminino,Solteiro(a)
4,8,1,3,1,Gravataí,RS,B,B,A,A,51-60,Masculino,Divorciado(a)/Desquitado(a)/Separado(a)


In [ ]:
mapa_cor_raca = {
    0: "Não declarado",
    1: "Branca",
    2: "Preta",
    3: "Parda",
    4: "Amarela",
    5: "Indígena",
    6: "Não dispõe da informação"
}

df_participantes["cor_raca_labels"] = df_participantes["cor_raca"].map(mapa_cor_raca)

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels,sexo_labels,estado_civil_labels,cor_raca_labels
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,18-20,Feminino,Solteiro(a),Branca
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30,Feminino,Solteiro(a),Branca
2,5,0,1,1,Sarandi,RS,H,F,J,A,26-30,Feminino,Solteiro(a),Branca
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,18-20,Feminino,Solteiro(a),Parda
4,8,1,3,1,Gravataí,RS,B,B,A,A,51-60,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),Branca


In [ ]:
def mapear_escolaridade(x):
    if x == "A":
        return "Nunca estudou"
    elif x == "B":
        return "Fundamental I incompleto"
    elif x == "C":
        return "Fundamental I completo, mas não Fundamental II"
    elif x == "D":
        return "Fundamental II completo, mas não Médio"
    elif x == "E":
        return "Médio completo"
    elif x == "F":
        return "Superior completo"
    elif x == "G":
        return "Pós-graduação"
    else:  # H
        return "Não sei"

# Criar a coluna com labels
df_participantes["escolaridade_pai_labels"] = df_participantes["q_escolaridade_pai"].apply(mapear_escolaridade)
df_participantes["escolaridade_mae_labels"] = df_participantes["q_escolaridade_mae"].apply(mapear_escolaridade)

# Criar um dicionário para mapear labels → números
mapa_escolaridade = {
    "Nunca estudou": 0,
    "Fundamental I incompleto": 1,
    "Fundamental I completo, mas não Fundamental II": 2,
    "Fundamental II completo, mas não Médio": 3,
    "Médio completo": 4,
    "Superior completo": 5,
    "Pós-graduação": 6,
    "Não sei": -1
}

# Criar colunas numéricas
df_participantes["escolaridade_pai"] = df_participantes["escolaridade_pai_labels"].map(mapa_escolaridade)
df_participantes["escolaridade_mae"] = df_participantes["escolaridade_mae_labels"].map(mapa_escolaridade)

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels,sexo_labels,estado_civil_labels,cor_raca_labels,escolaridade_pai_labels,escolaridade_mae_labels,escolaridade_pai,escolaridade_mae
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,18-20,Feminino,Solteiro(a),Branca,Superior completo,Superior completo,5,5
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30,Feminino,Solteiro(a),Branca,Fundamental I incompleto,"Fundamental I completo, mas não Fundamental II",1,2
2,5,0,1,1,Sarandi,RS,H,F,J,A,26-30,Feminino,Solteiro(a),Branca,Não sei,Superior completo,-1,5
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,18-20,Feminino,Solteiro(a),Parda,Fundamental I incompleto,Fundamental I incompleto,1,1
4,8,1,3,1,Gravataí,RS,B,B,A,A,51-60,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),Branca,Fundamental I incompleto,Fundamental I incompleto,1,1


In [ ]:
mapa_renda = {
    "A": (0, "Nenhuma renda"),
    "B": (1, "Muito baixa (até 2 SM)"),
    "C": (1, "Muito baixa (até 2 SM)"),
    "D": (1, "Muito baixa (até 2 SM)"),
    "E": (2, "Baixa (2-4 SM)"),
    "F": (2, "Baixa (2-4 SM)"),
    "G": (2, "Baixa (2-4 SM)"),
    "H": (3, "Média-baixa (4-7 SM)"),
    "I": (3, "Média-baixa (4-7 SM)"),
    "J": (3, "Média-baixa (4-7 SM)"),
    "K": (4, "Média (7-10 SM)"),
    "L": (4, "Média (7-10 SM)"),
    "M": (4, "Média (7-10 SM)"),
    "N": (5, "Média-alta (10-15 SM)"),
    "O": (5, "Média-alta (10-15 SM)"),
    "P": (6, "Alta (15-20 SM)"),
    "Q": (7, "Muito alta (20+ SM)")
}

df_participantes["renda_familiar"] = df_participantes["q_renda_familiar"].map(lambda x: mapa_renda[x][0])
df_participantes["renda_familiar_labels"] = df_participantes["q_renda_familiar"].map(lambda x: mapa_renda[x][1])

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,faixa_etaria_labels,sexo_labels,estado_civil_labels,cor_raca_labels,escolaridade_pai_labels,escolaridade_mae_labels,escolaridade_pai,escolaridade_mae,renda_familiar,renda_familiar_labels
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,18-20,Feminino,Solteiro(a),Branca,Superior completo,Superior completo,5,5,1,Muito baixa (até 2 SM)
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,26-30,Feminino,Solteiro(a),Branca,Fundamental I incompleto,"Fundamental I completo, mas não Fundamental II",1,2,2,Baixa (2-4 SM)
2,5,0,1,1,Sarandi,RS,H,F,J,A,26-30,Feminino,Solteiro(a),Branca,Não sei,Superior completo,-1,5,3,Média-baixa (4-7 SM)
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,18-20,Feminino,Solteiro(a),Parda,Fundamental I incompleto,Fundamental I incompleto,1,1,1,Muito baixa (até 2 SM)
4,8,1,3,1,Gravataí,RS,B,B,A,A,51-60,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),Branca,Fundamental I incompleto,Fundamental I incompleto,1,1,0,Nenhuma renda


In [ ]:
mapa_escola_em = {
    "A": (1, "Somente escola pública"),
    "B": (3, "Escola pública + privada (sem bolsa)"),
    "C": (2, "Escola pública + privada (com bolsa)"),
    "D": (5, "Somente escola privada (sem bolsa)"),
    "E": (4, "Somente escola privada (com bolsa)"),
    "F": (0, "Não frequentou EM")
}

df_participantes["tipo_escola_em"] = df_participantes["q_tipo_em"].map(lambda x: mapa_escola_em[x][0])
df_participantes["tipo_escola_em_labels"] = df_participantes["q_tipo_em"].map(lambda x: mapa_escola_em[x][1])

df_participantes.head(5)

,faixa_etaria,sexo,estado_civil,cor_raca,municipio_prova,uf_prova,q_escolaridade_pai,q_escolaridade_mae,q_renda_familiar,q_tipo_em,...,estado_civil_labels,cor_raca_labels,escolaridade_pai_labels,escolaridade_mae_labels,escolaridade_pai,escolaridade_mae,renda_familiar,renda_familiar_labels,tipo_escola_em,tipo_escola_em_labels
0,3,0,1,1,Porto Alegre,RS,F,F,C,A,...,Solteiro(a),Branca,Superior completo,Superior completo,5,5,1,Muito baixa (até 2 SM),1,Somente escola pública
1,5,0,1,1,São Luiz Gonzaga,RS,B,C,E,A,...,Solteiro(a),Branca,Fundamental I incompleto,"Fundamental I completo, mas não Fundamental II",1,2,2,Baixa (2-4 SM),1,Somente escola pública
2,5,0,1,1,Sarandi,RS,H,F,J,A,...,Solteiro(a),Branca,Não sei,Superior completo,-1,5,3,Média-baixa (4-7 SM),1,Somente escola pública
3,3,0,1,3,Novo Hamburgo,RS,B,B,C,A,...,Solteiro(a),Parda,Fundamental I incompleto,Fundamental I incompleto,1,1,1,Muito baixa (até 2 SM),1,Somente escola pública
4,8,1,3,1,Gravataí,RS,B,B,A,A,...,Divorciado(a)/Desquitado(a)/Separado(a),Branca,Fundamental I incompleto,Fundamental I incompleto,1,1,0,Nenhuma renda,1,Somente escola pública


In [ ]:
df_participantes = df_participantes.drop(columns=['q_escolaridade_pai', 'q_escolaridade_mae', 'q_renda_familiar', 'q_tipo_em'])

ordem_colunas_ml = ['faixa_etaria', 'faixa_etaria_labels', 'sexo', 'sexo_labels', 'estado_civil', 'estado_civil_labels',
         'cor_raca', 'cor_raca_labels', 'municipio_prova', 'uf_prova', 'escolaridade_pai', 'escolaridade_pai_labels',
         'escolaridade_mae', 'escolaridade_mae_labels', 'renda_familiar', 'renda_familiar_labels', 'tipo_escola_em', 'tipo_escola_em_labels']

ordem_colunas_dash = ['faixa_etaria_labels', 'sexo_labels', 'estado_civil_labels', 'cor_raca_labels', 'municipio_prova', 'uf_prova',
                      'escolaridade_pai_labels', 'escolaridade_mae_labels', 'renda_familiar_labels', 'tipo_escola_em_labels']

df_participantes = df_participantes[ordem_colunas_ml]
df_participantes_dash = df_participantes[ordem_colunas_dash]

df_participantes.head(5)

,faixa_etaria,faixa_etaria_labels,sexo,sexo_labels,estado_civil,estado_civil_labels,cor_raca,cor_raca_labels,municipio_prova,uf_prova,escolaridade_pai,escolaridade_pai_labels,escolaridade_mae,escolaridade_mae_labels,renda_familiar,renda_familiar_labels,tipo_escola_em,tipo_escola_em_labels
0,3,18-20,0,Feminino,1,Solteiro(a),1,Branca,Porto Alegre,RS,5,Superior completo,5,Superior completo,1,Muito baixa (até 2 SM),1,Somente escola pública
1,5,26-30,0,Feminino,1,Solteiro(a),1,Branca,São Luiz Gonzaga,RS,1,Fundamental I incompleto,2,"Fundamental I completo, mas não Fundamental II",2,Baixa (2-4 SM),1,Somente escola pública
2,5,26-30,0,Feminino,1,Solteiro(a),1,Branca,Sarandi,RS,-1,Não sei,5,Superior completo,3,Média-baixa (4-7 SM),1,Somente escola pública
3,3,18-20,0,Feminino,1,Solteiro(a),3,Parda,Novo Hamburgo,RS,1,Fundamental I incompleto,1,Fundamental I incompleto,1,Muito baixa (até 2 SM),1,Somente escola pública
4,8,51-60,1,Masculino,3,Divorciado(a)/Desquitado(a)/Separado(a),1,Branca,Gravataí,RS,1,Fundamental I incompleto,1,Fundamental I incompleto,0,Nenhuma renda,1,Somente escola pública


In [ ]:
df_participantes = df_participantes.sort_values(
    by=["uf_prova", "municipio_prova"]
).reset_index(drop=True)

df_participantes.head(5)

,faixa_etaria,faixa_etaria_labels,sexo,sexo_labels,estado_civil,estado_civil_labels,cor_raca,cor_raca_labels,municipio_prova,uf_prova,escolaridade_pai,escolaridade_pai_labels,escolaridade_mae,escolaridade_mae_labels,renda_familiar,renda_familiar_labels,tipo_escola_em,tipo_escola_em_labels
0,2,17,1,Masculino,1,Solteiro(a),3,Parda,Acrelândia,AC,-1,Não sei,2,"Fundamental I completo, mas não Fundamental II",1,Muito baixa (até 2 SM),1,Somente escola pública
1,3,18-20,0,Feminino,1,Solteiro(a),3,Parda,Acrelândia,AC,1,Fundamental I incompleto,4,Médio completo,1,Muito baixa (até 2 SM),1,Somente escola pública
2,4,21-25,1,Masculino,1,Solteiro(a),3,Parda,Acrelândia,AC,0,Nunca estudou,1,Fundamental I incompleto,1,Muito baixa (até 2 SM),1,Somente escola pública
3,4,21-25,0,Feminino,1,Solteiro(a),3,Parda,Acrelândia,AC,4,Médio completo,4,Médio completo,1,Muito baixa (até 2 SM),1,Somente escola pública
4,3,18-20,0,Feminino,1,Solteiro(a),3,Parda,Acrelândia,AC,2,"Fundamental I completo, mas não Fundamental II",4,Médio completo,1,Muito baixa (até 2 SM),1,Somente escola pública


In [ ]:
df_participantes_dash = df_participantes_dash.sort_values(
    by=["uf_prova", "municipio_prova"]
).reset_index(drop=True)

df_participantes_dash.head(5)

,faixa_etaria_labels,sexo_labels,estado_civil_labels,cor_raca_labels,municipio_prova,uf_prova,escolaridade_pai_labels,escolaridade_mae_labels,renda_familiar_labels,tipo_escola_em_labels
0,17,Masculino,Solteiro(a),Parda,Acrelândia,AC,Não sei,"Fundamental I completo, mas não Fundamental II",Muito baixa (até 2 SM),Somente escola pública
1,18-20,Feminino,Solteiro(a),Parda,Acrelândia,AC,Fundamental I incompleto,Médio completo,Muito baixa (até 2 SM),Somente escola pública
2,21-25,Masculino,Solteiro(a),Parda,Acrelândia,AC,Nunca estudou,Fundamental I incompleto,Muito baixa (até 2 SM),Somente escola pública
3,21-25,Feminino,Solteiro(a),Parda,Acrelândia,AC,Médio completo,Médio completo,Muito baixa (até 2 SM),Somente escola pública
4,18-20,Feminino,Solteiro(a),Parda,Acrelândia,AC,"Fundamental I completo, mas não Fundamental II",Médio completo,Muito baixa (até 2 SM),Somente escola pública


In [ ]:
df_resultados.head(5)

,NU_SEQUENCIAL,NU_ANO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,...,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,206403,2024,23052929.0,2301406.0,Aratuba,23.0,CE,2.0,1.0,1.0,...,CECEBEBCDBADDEBBABCDCAECEDADBAEABEADCEDADACBC,CAAAECDDDAECBECEDDCBDEDDCECBDCBCEADBBDBDDCBEDA...,CECEBEDADCAADECDBBCEBDCCCACABBABBADDDCEADBBCE,1.0,80.0,60.0,60.0,80.0,20.0,300.0
1,3604651,2024,42103770.0,4218004.0,Tijucas,42.0,SC,4.0,1.0,1.0,...,DADABCDCECEDEBBBEBCDBADACBCADCEBEADBAECAECEDA,AACEADDDACDADECBBDBEEBDDECCBEDDCADBBDEBCEBDCEC...,DBCECEACADABBECBEDADCCDCCABBBCECEBCEADABBADDD,1.0,160.0,200.0,200.0,180.0,180.0,920.0
2,1461268,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,CAEDBADBEBCCEDADBAEACBCADCEDEBBEBACECEABCDDAD,EAAACDDCDADDCBDEECEECBDCDADEEBCBEBADBCEBDBDDDD...,ABBABBADDDBBCECEADCEBCCCDBADCBEDAECADACDBCECE,1.0,120.0,120.0,40.0,120.0,80.0,480.0
3,4301058,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ACBCADCEDBAECEDADBADBEBCDADABCDEABCAEDEBBCECE,AACEADACDDCBEDADEEBDDBDBBCEECBDCBADDDCBDEDDCEC...,CEBCEADBBCEADDDABBABBCCDCCECEDBACADECABEDADCB,1.0,140.0,200.0,160.0,160.0,80.0,740.0
4,3148322,2024,21150354.0,2100436.0,Alto Alegre do Maranhão,21.0,MA,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_resultados = df_resultados.drop(columns=['NU_SEQUENCIAL', 'NU_ANO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
                                            'NO_MUNICIPIO_ESC',	'SG_UF_ESC',	'TP_DEPENDENCIA_ADM_ESC',	'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_PRESENCA_CN',
                                            'TP_PRESENCA_CH',	'TP_PRESENCA_LC','TP_PRESENCA_MT',	'CO_PROVA_CN',	'CO_PROVA_CH',	'CO_PROVA_LC',
                                            'CO_PROVA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT',
                                            'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT',
                                            'TP_STATUS_REDACAO',	'NU_NOTA_COMP1',	'NU_NOTA_COMP2',	'NU_NOTA_COMP3',	'NU_NOTA_COMP4',	'NU_NOTA_COMP5'])
df_resultados.head(5)

,NO_MUNICIPIO_PROVA,SG_UF_PROVA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
0,Aratuba,CE,436.8,377.8,423.4,427.1,300.0
1,Tijucas,SC,521.9,601.9,605.5,689.2,920.0
2,Rolândia,PR,363.0,548.4,557.2,456.4,480.0
3,Osasco,SP,550.7,553.8,605.9,629.1,740.0
4,São Mateus do Maranhão,MA,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_resultados = df_resultados.rename(columns={
    "NO_MUNICIPIO_PROVA": "municipio_prova",
    "SG_UF_PROVA": "uf_prova",
    "NU_NOTA_CN": "nota_ciencias_natureza",
    "NU_NOTA_CH": "nota_ciencias_humanas",
    "NU_NOTA_LC": "nota_linguagens_codigos",
    "NU_NOTA_MT": "nota_matematica",
    "NU_NOTA_REDACAO": "nota_redacao"
})

df_resultados.head(5)

,municipio_prova,uf_prova,nota_ciencias_natureza,nota_ciencias_humanas,nota_linguagens_codigos,nota_matematica,nota_redacao
0,Aratuba,CE,436.8,377.8,423.4,427.1,300.0
1,Tijucas,SC,521.9,601.9,605.5,689.2,920.0
2,Rolândia,PR,363.0,548.4,557.2,456.4,480.0
3,Osasco,SP,550.7,553.8,605.9,629.1,740.0
4,São Mateus do Maranhão,MA,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_resultados = df_resultados.sort_values(
    by=["uf_prova", "municipio_prova"]
).reset_index(drop=True)

df_resultados.head(5)

,municipio_prova,uf_prova,nota_ciencias_natureza,nota_ciencias_humanas,nota_linguagens_codigos,nota_matematica,nota_redacao
0,Acrelândia,AC,NaN,NaN,NaN,NaN,NaN
1,Acrelândia,AC,NaN,NaN,NaN,NaN,NaN
2,Acrelândia,AC,443.8,448.4,447.5,559.5,380.0
3,Acrelândia,AC,531.8,553.6,553.7,532.2,560.0
4,Acrelândia,AC,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_resultados = df_resultados.drop(columns=['municipio_prova', 'uf_prova'])

df_resultados.head(5)

,nota_ciencias_natureza,nota_ciencias_humanas,nota_linguagens_codigos,nota_matematica,nota_redacao
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,443.8,448.4,447.5,559.5,380.0
3,531.8,553.6,553.7,532.2,560.0
4,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_enem_2024_dash = pd.concat([df_participantes_dash, df_resultados], axis=1)

df_enem_2024_dash.head(5)

,faixa_etaria_labels,sexo_labels,estado_civil_labels,cor_raca_labels,municipio_prova,uf_prova,escolaridade_pai_labels,escolaridade_mae_labels,renda_familiar_labels,tipo_escola_em_labels,nota_ciencias_natureza,nota_ciencias_humanas,nota_linguagens_codigos,nota_matematica,nota_redacao
0,17,Masculino,Solteiro(a),Parda,Acrelândia,AC,Não sei,"Fundamental I completo, mas não Fundamental II",Muito baixa (até 2 SM),Somente escola pública,NaN,NaN,NaN,NaN,NaN
1,18-20,Feminino,Solteiro(a),Parda,Acrelândia,AC,Fundamental I incompleto,Médio completo,Muito baixa (até 2 SM),Somente escola pública,NaN,NaN,NaN,NaN,NaN
2,21-25,Masculino,Solteiro(a),Parda,Acrelândia,AC,Nunca estudou,Fundamental I incompleto,Muito baixa (até 2 SM),Somente escola pública,443.8,448.4,447.5,559.5,380.0
3,21-25,Feminino,Solteiro(a),Parda,Acrelândia,AC,Médio completo,Médio completo,Muito baixa (até 2 SM),Somente escola pública,531.8,553.6,553.7,532.2,560.0
4,18-20,Feminino,Solteiro(a),Parda,Acrelândia,AC,"Fundamental I completo, mas não Fundamental II",Médio completo,Muito baixa (até 2 SM),Somente escola pública,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_enem_2024_dash.to_csv("enem_2024_dash.csv", index=False, sep=";", encoding="latin1")

In [ ]:
def amostrar_df(df, frac=0.03, random_state=42):
    """
    Retorna uma amostra estratificada do DataFrame,
    considerando 'uf_prova' e 'municipio_prova' proporcionalmente.
    """
    return (
        df.groupby(["uf_prova", "municipio_prova"], group_keys=False)
          .apply(lambda x: x.sample(frac=frac, random_state=random_state))
    )

In [ ]:
df_amostra = amostrar_df(df_enem_2024_dash, frac=0.036)

156005 linhas na amostra
uf_prova
SP    0.149418
MG    0.090952
BA    0.086972
RJ    0.066774
RS    0.064376
Name: proportion, dtype: float64


/tmp/ipython-input-1859711017.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=frac, random_state=random_state))


In [ ]:
df_amostra.to_csv("enem_2024_dash_amostra.csv", index=False, sep=";", encoding="latin1")